In [2]:
import sys
import numpy as np
from keras.datasets import mnist

In [3]:
(x_train, y_train),(x_test,y_test) = mnist.load_data()

In [4]:
images, labels = (x_train[0:1000].reshape(1000,28*28) / 255, y_train[0:1000])

In [5]:
one_hot_labels = np.zeros((len(labels),10))

In [6]:
for i,l in enumerate(labels):
    one_hot_labels[i][l] = 1
labels = one_hot_labels

In [7]:
test_images = x_test.reshape(len(x_test),28*28) /255
test_labels = np.zeros((len(y_test),10))
for i,l in enumerate(y_test):
    test_labels[i][l] = 1

In [8]:
np.random.seed(1)

In [9]:
relu = lambda x:(x>0) * x
relu2deriv = lambda x: x>=0

In [10]:
alpha = 0.005
iterations = 350
hidden_size = 40
pixels_per_image = 784
num_labels = 10

In [11]:
weights_0_1 = 0.2*np.random.random((pixels_per_image,hidden_size)) - 0.1
weights_1_2 = 0.2*np.random.random((hidden_size,num_labels)) - 0.1

In [12]:
for j in range(iterations):
    error, correct_cnt = (0.0, 0)
    
    for i in range(len(images)):
        layer_0 = images[i:i+1]
        layer_1 = relu(np.dot(layer_0,weights_0_1))
        layer_2 = np.dot(layer_1,weights_1_2)

        error += np.sum((labels[i:i+1] - layer_2) ** 2)
        correct_cnt += int(np.argmax(layer_2) == \
                                        np.argmax(labels[i:i+1]))

        layer_2_delta = (labels[i:i+1] - layer_2)
        layer_1_delta = layer_2_delta.dot(weights_1_2.T)\
                                    * relu2deriv(layer_1)
        weights_1_2 += alpha * layer_1.T.dot(layer_2_delta)
        weights_0_1 += alpha * layer_0.T.dot(layer_1_delta)

    sys.stdout.write("\r I:"+str(j)+ \
                     " Train-Err:" + str(error/float(len(images)))[0:5] +\
                     " Train-Acc:" + str(correct_cnt/float(len(images))))

 I:349 Train-Err:0.108 Train-Acc:1.099

In [13]:
if(j%10 == 0 or j == iterations-1):
    error , correct_cnt = (0.0,0)
    
    for i in range(len(test_images)):
        layer_0 = test_images[i:i+1]
        layer_1 = relu(np.dot(layer_0,weights_0_1))
        layer_2 = np.dot(layer_1,weights_1_2)
        
        error += np.sum((test_labels[i:i+1] - layer_2) ** 2)
        
        correct_cnt += int(np.argmax(layer_2)== np.argmax(test_labels[i:i+1]))
        
    sys.stdout.write(" Test-Err:" + str(error/float(len(test_images)))[0:5] +\
                     " Test-Acc:" + str(correct_cnt/float(len(test_images))) + "\n")
    print()

 Test-Err:0.653 Test-Acc:0.7073



## Dropout in code

In [14]:
i =0
layer_0 = images[i:i+1]
dropout_mask = np.random.randint(2,size=layer_1.shape)

layer_1 += dropout_mask * 2
layer_2 = np.dot(layer_1,weights_1_2)

error += np.sum((labels[i:i+1]-layer_2)**2)

correct_cnt += int(np.argmax(layer_2) == np.argmax(labels[i+i+1]))

layer_2_delta = (labels[i:i+1] - layer_2)
layer_1_delta = layer_2_delta.dot(weights_1_2.T) * relu2deriv(layer_1)

layer_1_delta *= dropout_mask

weights_1_2 += alpha * layer_1.T.dot(layer_2_delta)
weights_0_1 += alpha * layer_0.T.dot(layer_1_delta)

In [15]:
import numpy,sys
np.random.seed(1)

def relu(x):
    return (x>=0)*x
def reluderiv(x):
    return x>=0


In [21]:
alpha = 0.005
iterations = 300
hidden_size = 100
pixels_per_image = 784
num_labels = 10

In [22]:
weights_0_1 = 0.2*np.random.random((pixels_per_image,hidden_size)) - 0.1
weights_1_2 = 0.2*np.random.random((hidden_size,num_labels)) - 0.1

In [27]:
for j in range(iterations):
    error, correct_cnt = (0.0,0)
    
    for i in range(len(images)):
        layer_0 = images[i:i+1]
        layer_1 = relu(np.dot(layer_0,weights_0_1))
        dropout_mask = np.random.randint(2, size=layer_1.shape)
        layer_1 *= dropout_mask * 2
        layer_2 = np.dot(layer_1,weights_1_2)
        
        error += np.sum((labels[i:i+1]-layer_2)**2)
        correct_cnt += int(np.argmax(layer_2) == np.argmax(labels[i:i+1]))
        
        layer_2_delta = (labels[i:i+1]-layer_2)
        layer_1_delta = layer_2_delta.dot(weights_1_2.T)*reluderiv(layer_1)
        layer_1_delta *= dropout_mask
        
        weights_1_2 += alpha * layer_1.T.dot(layer_2_delta)
        weights_0_1 += alpha * layer_0.T.dot(layer_1_delta)

    if(j%10 == 0):
        test_error = 0.0
        test_correct_cnt = 0

        for i in range(len(test_images)):
            layer_0 = test_images[i:i+1]
            layer_1 = relu(np.dot(layer_0,weights_0_1))
            layer_2 = np.dot(layer_1, weights_1_2)

            test_error += np.sum((test_labels[i:i+1] - layer_2) ** 2)
            test_correct_cnt += int(np.argmax(layer_2) == np.argmax(test_labels[i:i+1]))

        sys.stdout.write("\n" + \
                         "I:" + str(j) + \
                         " Test-Err:" + str(test_error/ float(len(test_images)))[0:5] +\
                         " Test-Acc:" + str(test_correct_cnt/ float(len(test_images)))+\
                         " Train-Err:" + str(error/ float(len(images)))[0:5] +\
                         " Train-Acc:" + str(correct_cnt/ float(len(images))))


I:0 Test-Err:0.636 Test-Acc:0.6376 Train-Err:0.901 Train-Acc:0.39
I:10 Test-Err:0.460 Test-Acc:0.8015 Train-Err:0.480 Train-Acc:0.764
I:20 Test-Err:0.440 Test-Acc:0.8152 Train-Err:0.444 Train-Acc:0.815
I:30 Test-Err:0.430 Test-Acc:0.8225 Train-Err:0.428 Train-Acc:0.821
I:40 Test-Err:0.418 Test-Acc:0.8281 Train-Err:0.411 Train-Acc:0.842
I:50 Test-Err:0.404 Test-Acc:0.824 Train-Err:0.380 Train-Acc:0.841
I:60 Test-Err:0.407 Test-Acc:0.822 Train-Err:0.383 Train-Acc:0.846
I:70 Test-Err:0.396 Test-Acc:0.8232 Train-Err:0.369 Train-Acc:0.868
I:80 Test-Err:0.393 Test-Acc:0.822 Train-Err:0.346 Train-Acc:0.882
I:90 Test-Err:0.389 Test-Acc:0.825 Train-Err:0.351 Train-Acc:0.876
I:100 Test-Err:0.392 Test-Acc:0.8248 Train-Err:0.341 Train-Acc:0.891
I:110 Test-Err:0.390 Test-Acc:0.8213 Train-Err:0.340 Train-Acc:0.886
I:120 Test-Err:0.386 Test-Acc:0.8222 Train-Err:0.341 Train-Acc:0.882
I:130 Test-Err:0.397 Test-Acc:0.8157 Train-Err:0.340 Train-Acc:0.887
I:140 Test-Err:0.396 Test-Acc:0.815 Train-Err:0.3